# EPAM Python S2 Closures Decorators and I/O

## Namespaces and Scopes

### Namespaces

A namespace is a collection of currently defined symbolic names along with information about the object that each name references: **a dictionary in which the keys are the object names and the values are the objects themselves**

4 types of namespaces:
- Built-In
- Global
- Enclosing
- Local

The **built-in** namespace contains the names of all of python's built-in objects 

In [3]:
print(dir(__builtins__))

['ArithmeticError', 'AssertionError', 'AttributeError', 'BaseException', 'BlockingIOError', 'BrokenPipeError', 'BufferError', 'BytesWarning', 'ChildProcessError', 'ConnectionAbortedError', 'ConnectionError', 'ConnectionRefusedError', 'ConnectionResetError', 'DeprecationWarning', 'EOFError', 'Ellipsis', 'EnvironmentError', 'Exception', 'False', 'FileExistsError', 'FileNotFoundError', 'FloatingPointError', 'FutureWarning', 'GeneratorExit', 'IOError', 'ImportError', 'ImportWarning', 'IndentationError', 'IndexError', 'InterruptedError', 'IsADirectoryError', 'KeyError', 'KeyboardInterrupt', 'LookupError', 'MemoryError', 'ModuleNotFoundError', 'NameError', 'None', 'NotADirectoryError', 'NotImplemented', 'NotImplementedError', 'OSError', 'OverflowError', 'PendingDeprecationWarning', 'PermissionError', 'ProcessLookupError', 'RecursionError', 'ReferenceError', 'ResourceWarning', 'RuntimeError', 'RuntimeWarning', 'StopAsyncIteration', 'StopIteration', 'SyntaxError', 'SyntaxWarning', 'SystemError

**Global** namespace contains any names defined at the level of the main program. Python interpreter creates the global namespace when the main program body starts, and it remains in existence until the interpreter terminates


In [4]:
type(globals())

dict

In [5]:
print(globals())

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', '# EPAM DAE S2 Python Session_3 ', '### Namespaces', 'print(dir(__builtins__))', 'type(globals())', 'print(globals())'], '_oh': {4: <class 'dict'>}, '_dh': [PosixPath('/home/goetie/python_stuff/Courses/EPAM S2')], 'In': ['', '# EPAM DAE S2 Python Session_3 ', '### Namespaces', 'print(dir(__builtins__))', 'type(globals())', 'print(globals())'], 'Out': {4: <class 'dict'>}, 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7fd2440da5b0>>, 'exit': <IPython.core.autocall.ZMQExitAutocall object at 0x7fd2440f23d0>, 'quit': <IPython.core.autocall.ZMQExitAutocall object at 0x7fd2440f23d0>, '_': <class 'dict'>, '__': '', '___': '', '_i': 'type(globals())', '_ii': 'prin

The **Local and Enclosing** namespaces are created whenever a function executes. These namespaces are local and only exist within a function, they exist until the termination of a function

**NB: Functions do not exist independently from one another only at the level of the main program. You can also define one function inside another**

In [7]:
def f():
    print('Start f()')
    def g():
        print('Start g()')
        print('End g()') 
    g()
    print('End f()')
f()

Start f()
Start g()
End g()
End f()


The namespace created automatically for g() is the local namespace, and the namespace created for f() is the enclosing namespace

In [12]:
def f(x,y):
    s = 'foo'
    print(locals())
f(10, 0.5)

{'x': 10, 'y': 0.5, 's': 'foo'}


### Scope

Suppose you refer to the name x in your code and x exists in several nmespaces. How does python know which one you mean?

The answer is in the concept of **scope**

The scope of a name is the region of a program in which that name has meaning.

The interpreter determines this at runtime based on where the name definition occurs and where in the code the name is referenced

Scope types are named the same as namespaces:
- Local scope: if you refer to x inside a function, then the interpreter first searches for it in the innermost scope thats local to that functions
- Enclosing scope: if x isn't in the local scope but appears in a function that resides inside another function, then the interpreter searches in the enclosing function's scope
- Global scope: if neither of the above searches succeeded, then the interpreter looks in the global scope
- Built-in scope: if it can't find x anywhere else, then the interpreter tries the built-in scope

#### The LEGB rule - Local-Enclosing-Global-Builtin

**Global** keyword:

In [24]:
x = 20
def f():
    # change the global variable in the local namespace - global() keyword
    global x
    x = 40
    y = 10
    print(x)
    def g():
        nonlocal y
        print(y)
    g()
f()

40
10


## Closures

A closure causes the inner function to retain the state of its environment when called. The closure isnt the inner function itself but the inner function along with its enclosing environment. The closure captures the local variables and name in the containing function and keeps them around

A **closure** is a nested function that allows us to access variables of the outer function even after the outer function is closed

#### A closure is an inner function with an extended scope that encompasses nonlocal variables of the outer function. So it remembers the nonlocal variables in the enclosing scopes even if they are not present in memory.

In [39]:
def maker(n):
    def action(x):
        return x*n
    return action
f = maker(2)
g = maker(3)
f, g, g.__name__

(<function __main__.maker.<locals>.action(x)>,
 <function __main__.maker.<locals>.action(x)>,
 'action')

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*fxPw-BUfHoPcGkIBBWTgHQ.jpeg)

In [29]:
f(3), g(3)

(6, 9)

In [55]:
def f(x):
    z = 2
    def g(y):
        return z*x + y
    return g
a = 2
F = f(a)
F(10), F.__code__.co_freevars, F.__closure__[0].cell_contents

(14, ('x', 'z'), 2)

![img](https://miro.medium.com/v2/resize:fit:720/format:webp/1*zjrkiqxMi7Dy5aFL2OkawQ.jpeg)

#### Deploy closures using lambda functions:

In [56]:
def f(x):
    z = 2
    return lambda y: z*x+y
a = 5
b = 1
f(a)(b)

11

To define a closure we need an inner function that:

- It should be returned by the outer function.

- it should capture some of the nonlocal variables of the outer function. This can be done by accessing those variables, or defining them as a nonlocal variable or having a nested closure that needs to capture them.

After defining the closure, to initialize it, you have to call the outer function to return the closure.

In functional programming, closures make it possible to bind data to a function without actually passing them as parameters. This is similar to what a class does in object-oriented programming.

In [58]:
class NthRoot:
    def __init__(self, n=2):
        self.n = n
    def set_root(n):
        self.n = n
    def calc(self, x):
        return x ** (1/self.n)
thirdRoot = NthRoot(3)
print(thirdRoot.calc(27))
def nth_root(n=2):
    def calc(x):
        return x ** (1/n)
    return calc
third_root = nth_root(3)
print(third_root(27))

3.0
3.0


As you see the outer function can play the role of a constructor for us here. It initializes the nonlocal variables that will be used by the inner function. However, there are also some differences. The NthRoot class can have more methods that can be invoked by the object thirdRoot . However, what is returned by nth_root is a function itself. So this method is more limited compared to what classes can do.

#### Implement composition using a closure:

In [62]:
def compose(g, f):
    def h(*args, **kwargs):
        return g(f(*args, **kwargs))
    return h
# example: inch-to-foot to foot-to-meter
inch_to_foot= lambda x: x/12
foot_meter= lambda x: x * 0.3048
inch_to_meter = compose(foot_meter, inch_to_foot)
inch_to_meter(12)

0.3048

## Decorators

Decorating is a way to manage functions and classes

***Function decorators*** bound function name with another callable object; **this is done when function is defined**.

Function decorators add an extra layer of logic which performs some actions when called

***Class decorators*** bound class name with another callable object; this is done when class is defined. Class decorators add an extra layer of logic which manages classes and instances which are created when class is referred

In [36]:
# dummy decorator
def null_decorator(func):
    return func

def greet():
    return 'hello'
# decorator call
greet = null_decorator(greet)
greet()
# the same as
@null_decorator
def greet():
    return 'hello'
greet()

'hello'

In [63]:
# To init a closure, we need to assign the result of the outer function to a new variable:
h=f(a)

In [68]:
# To init a decorator, we need to assign the same function to itself, but wrapped in a decorating function:
def wrapperfunc(f):
    pass

def F():
    pass
F = wrapperfunc(F)
# same as
@wrapperfunc
def F():
    pass

In [74]:
def deco(f):
    def g(*args, **kwargs):
        return f(*args, **kwargs)
    return g

#decorate func(x)
@deco
def func(x):
     return 2*x
func(2), func.__name__

(4, 'g')

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*bCL9EtwhR9IXndx2Q3iTxA.jpeg)

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*wql7OYmp7rTrt-yaO35MKQ.jpeg)

So to summarize it, after decoration, the variable func refers to the closure g, and inside g, f refers to the func(x) definition. In fact g is now acting as an interface for the original function func(x) which was decorated. We cannot directly call func(x)outside g. Instead, we first call func to call g, and then inside g we can call f to call the original function func(x). So we are calling the original function func(x) using the closure g.

#### Stacked decorators

In [77]:
def deco1(f):
    def g1(*args, **kwargs):
        print("Calling ", f.__name__, "using deco1")
        return f(*args, **kwargs)
    return g1
def deco2(f):
    def g2(*args, **kwargs):
        print("Calling ", f.__name__, "using deco2")
        return f(*args, **kwargs)
    return g2

@deco2 # same as deco2(deco1(func(x)))
@deco1
def func(x):
    return 2*x

func(2)

Calling  g1 using deco2
Calling  func using deco1


4

![](https://miro.medium.com/v2/resize:fit:720/format:webp/1*IdDXXpVOnM1wtoHgjRFX2Q.jpeg)

#### Parameterized decorators

In [78]:
def deco(msg_before, msg_after):
    def original_deco(f):
        def g(*args, **kwargs):
            print(msg_before + " " + f.__name__)
            result =  f(*args, **kwargs)
            print(msg_after + " " + f.__name__)
            return result
        return g
    return original_deco

@deco("Starting", "Finished")
def func(x):
    return 2*x

func(2)

Starting func
Finished func


4

In [81]:
import time

def paused(t):
    def parameterized_paused(f):
        def wrapper(*args, **kwargs):
            time.sleep(t)
            return f(*args, **kwargs)
        return wrapper
    return parameterized_paused

@paused(3) # similar to func = paused(3)(func)
def func(x,y):
    return x+y

func(1,2)

3

## Python I/O

In [ ]:
#f = open("file_name.txt")
#f.close()

`open() function`: open(
- file,
- mode = 'r',
- buffering = -1,
- encoding = None,
- errors = None,
- newline = None,
- closefd = True,
- opener = None

)

file modes:
- 'r' read-only
- 'w' write-only
- 'x' same as 'w' but raises an exception if the file already exists
- 'a' Appending
- 'b' binary mode
- 't' text mode (default)
- '+' read-write

Working with file line by line

In [88]:
# Writing
# f = open("file_name.txt", "w")
# f.write("first line\n")
# f.write("second line")
# f.close
# Reading
# f = open("file_name.txt")
# print(f.readline())
# or
# for line in f:
#    print(line)
# f.close

#### Binary files

In [89]:
# f = open("binary_file.dat", "bw")
# f.write(b"0123456789abcdef")
# f.close

### Working with files via context manager

In [92]:
# with open("file_name.txt") as
# my_file:
#    print(my_file.read())

## Python Modules

In [94]:
import math 
print(dir(math)) #check the namespace imported with the module

['__doc__', '__loader__', '__name__', '__package__', '__spec__', 'acos', 'acosh', 'asin', 'asinh', 'atan', 'atan2', 'atanh', 'ceil', 'comb', 'copysign', 'cos', 'cosh', 'degrees', 'dist', 'e', 'erf', 'erfc', 'exp', 'expm1', 'fabs', 'factorial', 'floor', 'fmod', 'frexp', 'fsum', 'gamma', 'gcd', 'hypot', 'inf', 'isclose', 'isfinite', 'isinf', 'isnan', 'isqrt', 'ldexp', 'lgamma', 'log', 'log10', 'log1p', 'log2', 'modf', 'nan', 'perm', 'pi', 'pow', 'prod', 'radians', 'remainder', 'sin', 'sinh', 'sqrt', 'tan', 'tanh', 'tau', 'trunc']


## Additionals: Callable() function

Collable objects in python:
- Functions
- Classes
- Methods (functions that hang off of classes)
- Instances of classes can be turned into callables

In [101]:
callable(enumerate)

True

In [102]:
type(range(1))

range